In [26]:
import pandas as pd
import numpy as np
import h5py
np.random.seed(42)
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import gensim.models
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Dropout, Activation, Embedding, Bidirectional
from keras.layers import LSTM, SimpleRNN, GRU
import keras.callbacks
import re
from nltk.tokenize import word_tokenize
from utils import clean_tweet, tokenize_tweet
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
##importer le word2vec google
WORD2VEC_VECTORS_BIN = 'C:\\Users\\PC\\python\\deep learning\\Projet\\GoogleNews-vectors-negative300.bin'

w2v = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_VECTORS_BIN, binary=True)

In [32]:
stop_words = set(stopwords.words('english'))
##lecture des donnes
data_clickbait=pd.read_csv('clickbait_data',sep='\n',header=None)
data_noclickbait=pd.read_csv('non_clickbait_data',sep='\n',header=None)

data_clickbait.insert(1,"class",np.ones(15999))
data_noclickbait.insert(1,"class",np.zeros(16001))

data_Final=pd.concat((data_clickbait,data_noclickbait),ignore_index=True) # concatenation des click vs nonclick data
df=data_Final.rename(columns={0:"text"}) # renommer la colonne 0
df=df.sample(frac=1).reset_index(drop=True) # faire un shuffle avec reset index ( index remit a zero)



In [33]:
##Data cleaning
df['cleaned_tweet'] = df['text'].apply(clean_tweet)

In [34]:
##separer les donnes de façon non aléatoire pour eviter d'avoir un probleme lors de la cross validation a cause des indexs

split=int(2*len(df)/3)

text_train,text_test,y_train,y_test=df.iloc[:split,2],df.iloc[split:,2],df.iloc[:split,1],df.iloc[split:,1]


In [35]:
## appliquer le word2vec de google mais Charactere par Charactere !!

dimsize=300
sequence_size=30

def compute_matrix(Text):
    X = np.zeros((len(Text), sequence_size, dimsize))
    for idx, review in enumerate(Text):
        sequence = np.zeros((sequence_size, dimsize))
        tokens = review
        count = 0
        for token in tokens:
            if count == sequence_size:
                break
            try:
                token = token.lower()
                sequence[count] = w2v[token]
                count += 1
            except:
                pass
        X[idx] = sequence

    return X


In [50]:
X_train=compute_matrix(text_train)
X_test=compute_matrix(text_test)

##creer le model 
def creat_model():
    
    model = Sequential()
    model.add(Bidirectional(GRU(64, dropout=0.3), input_shape=(sequence_size, dimsize)))  # try using a GRU instead, for fun
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='ADAM',metrics=['accuracy'])
    
    return model

In [38]:
# try using different optimizers and different optimizer configs
def fit_model(model,X_train,y_train,x_valid,y_valid,batch_size = 64):

    #earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')

    model.fit(X_train, y_train, batch_size=batch_size, epochs=20,validation_data=[x_valid,y_valid],callbacks=callback_list)
    #score, acc = model.evaluate(X_test, y_test,batch_size=batch_size)
    
    return model

In [39]:
## definir un fichier qui sauvegarde les poids du meilleur modele
cp=ModelCheckpoint("best_model.h5",verbose=1,save_best_only=True) 
callback_list=[cp] 

In [40]:
##cross_validation 10folds

n_folds = 10
skf = StratifiedKFold(n_splits=n_folds, shuffle=True)


for i, (train, valid) in enumerate(skf.split(X_train,y_train)):
    print(X_train[train].shape,y_train[train].shape,X_train[valid].shape,y_train[valid].shape)
    print ("Running Fold", i+1, "/", n_folds)
    model = None # Clearing the NN.
    model = creat_model()
    hist=fit_model(model, X_train[train], y_train[train], X_train[valid], y_train[valid])
    

(19199, 30, 300) (19199,) (2134, 30, 300) (2134,)
Running Fold 1 / 10
Train on 19199 samples, validate on 2134 samples
Epoch 1/20
19199/19199 [==============================] - 37s 2ms/step - loss: 0.4216 - accuracy: 0.8141 - val_loss: 0.3206 - val_accuracy: 0.8655

Epoch 00001: val_loss improved from inf to 0.32059, saving model to best_model.h5
Epoch 2/20
19199/19199 [==============================] - 35s 2ms/step - loss: 0.3162 - accuracy: 0.8715 - val_loss: 0.2917 - val_accuracy: 0.8843

Epoch 00002: val_loss improved from 0.32059 to 0.29175, saving model to best_model.h5
Epoch 3/20
19199/19199 [==============================] - 38s 2ms/step - loss: 0.2722 - accuracy: 0.8908 - val_loss: 0.2606 - val_accuracy: 0.8936

Epoch 00003: val_loss improved from 0.29175 to 0.26063, saving model to best_model.h5
Epoch 4/20
19199/19199 [==============================] - 37s 2ms/step - loss: 0.2428 - accuracy: 0.9031 - val_loss: 0.2429 - val_accuracy: 0.9067

Epoch 00004: val_loss improved from

19199/19199 [==============================] - 34s 2ms/step - loss: 0.0276 - accuracy: 0.9905 - val_loss: 0.2839 - val_accuracy: 0.9302

Epoch 00019: val_loss did not improve from 0.19355
Epoch 20/20
19199/19199 [==============================] - 36s 2ms/step - loss: 0.0209 - accuracy: 0.9933 - val_loss: 0.3119 - val_accuracy: 0.9278

Epoch 00020: val_loss did not improve from 0.19355
(19199, 30, 300) (19199,) (2134, 30, 300) (2134,)
Running Fold 3 / 10
Train on 19199 samples, validate on 2134 samples
Epoch 1/20
19199/19199 [==============================] - 42s 2ms/step - loss: 0.4131 - accuracy: 0.8169 - val_loss: 0.3224 - val_accuracy: 0.8735

Epoch 00001: val_loss did not improve from 0.19355
Epoch 2/20
19199/19199 [==============================] - 41s 2ms/step - loss: 0.3138 - accuracy: 0.8738 - val_loss: 0.2884 - val_accuracy: 0.8810

Epoch 00002: val_loss did not improve from 0.19355
Epoch 3/20
19199/19199 [==============================] - 40s 2ms/step - loss: 0.2719 - accurac

19200/19200 [==============================] - 72s 4ms/step - loss: 0.0287 - accuracy: 0.9903 - val_loss: 0.2607 - val_accuracy: 0.9353

Epoch 00019: val_loss did not improve from 0.18393
Epoch 20/20
19200/19200 [==============================] - 70s 4ms/step - loss: 0.0236 - accuracy: 0.9919 - val_loss: 0.2698 - val_accuracy: 0.9306

Epoch 00020: val_loss did not improve from 0.18393
(19200, 30, 300) (19200,) (2133, 30, 300) (2133,)
Running Fold 5 / 10
Train on 19200 samples, validate on 2133 samples
Epoch 1/20
19200/19200 [==============================] - 78s 4ms/step - loss: 0.4221 - accuracy: 0.8123 - val_loss: 0.3343 - val_accuracy: 0.8570

Epoch 00001: val_loss did not improve from 0.18393
Epoch 2/20
19200/19200 [==============================] - 76s 4ms/step - loss: 0.3121 - accuracy: 0.8743 - val_loss: 0.2996 - val_accuracy: 0.8776

Epoch 00002: val_loss did not improve from 0.18393
Epoch 3/20
19200/19200 [==============================] - 77s 4ms/step - loss: 0.2708 - accurac

19200/19200 [==============================] - 81s 4ms/step - loss: 0.0270 - accuracy: 0.9908 - val_loss: 0.3200 - val_accuracy: 0.9189

Epoch 00019: val_loss did not improve from 0.18393
Epoch 20/20
19200/19200 [==============================] - 81s 4ms/step - loss: 0.0218 - accuracy: 0.9928 - val_loss: 0.3218 - val_accuracy: 0.9231

Epoch 00020: val_loss did not improve from 0.18393
(19200, 30, 300) (19200,) (2133, 30, 300) (2133,)
Running Fold 7 / 10
Train on 19200 samples, validate on 2133 samples
Epoch 1/20
19200/19200 [==============================] - 90s 5ms/step - loss: 0.4149 - accuracy: 0.8158 - val_loss: 0.3266 - val_accuracy: 0.8697

Epoch 00001: val_loss did not improve from 0.18393
Epoch 2/20
19200/19200 [==============================] - 87s 5ms/step - loss: 0.3133 - accuracy: 0.8730 - val_loss: 0.2850 - val_accuracy: 0.8847

Epoch 00002: val_loss did not improve from 0.18393
Epoch 3/20
19200/19200 [==============================] - 93s 5ms/step - loss: 0.2696 - accurac

19200/19200 [==============================] - 61s 3ms/step - loss: 0.0354 - accuracy: 0.9880 - val_loss: 0.2360 - val_accuracy: 0.9330

Epoch 00018: val_loss did not improve from 0.16627
Epoch 19/20
19200/19200 [==============================] - 62s 3ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.2329 - val_accuracy: 0.9344

Epoch 00019: val_loss did not improve from 0.16627
Epoch 20/20
19200/19200 [==============================] - 65s 3ms/step - loss: 0.0201 - accuracy: 0.9938 - val_loss: 0.2578 - val_accuracy: 0.9381

Epoch 00020: val_loss did not improve from 0.16627
(19200, 30, 300) (19200,) (2133, 30, 300) (2133,)
Running Fold 9 / 10
Train on 19200 samples, validate on 2133 samples
Epoch 1/20
19200/19200 [==============================] - 75s 4ms/step - loss: 0.4253 - accuracy: 0.8121 - val_loss: 0.3420 - val_accuracy: 0.8608

Epoch 00001: val_loss did not improve from 0.16627
Epoch 2/20
19200/19200 [==============================] - 66s 3ms/step - loss: 0.3127 - accura

19200/19200 [==============================] - 74s 4ms/step - loss: 0.0333 - accuracy: 0.9882 - val_loss: 0.2261 - val_accuracy: 0.9316

Epoch 00018: val_loss did not improve from 0.16627
Epoch 19/20
19200/19200 [==============================] - 69s 4ms/step - loss: 0.0258 - accuracy: 0.9914 - val_loss: 0.2467 - val_accuracy: 0.9311

Epoch 00019: val_loss did not improve from 0.16627
Epoch 20/20
19200/19200 [==============================] - 71s 4ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.2662 - val_accuracy: 0.9376

Epoch 00020: val_loss did not improve from 0.16627


In [43]:

#sauvegarder les poids du meilleure model pour les utiliser dans le teste
hist.load_weights("best_model.h5")

##EVALUATION
score, acc = hist.evaluate(X_test, y_test,
                            batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)



10667/10667 [==============================] - 23s 2ms/step
Test score: 0.1946373532881629
Test accuracy: 0.9301584362983704


In [44]:
#prediction
y_pred = model.predict_classes(X_test)
y_scores = model.predict_proba(X_test)

##afficher les résultats de la prediction
roc = roc_auc_score(y_test, y_scores)
print('ROC score:', roc)



ROC score: 0.9775914470379724


In [45]:
metrics = classification_report(y_test, y_pred, digits=4)
print('Classification Report \n')
print(metrics)



Classification Report 

              precision    recall  f1-score   support

         0.0     0.9149    0.9484    0.9313      5328
         1.0     0.9465    0.9120    0.9289      5339

    accuracy                         0.9302     10667
   macro avg     0.9307    0.9302    0.9301     10667
weighted avg     0.9307    0.9302    0.9301     10667



In [46]:
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix \n')
print (cm)

Confusion Matrix 

[[5053  275]
 [ 470 4869]]
